In [1]:
import numpy as np
import scipy.signal as ss
import pywt

def auto2bpm(index, ln=10337):
    return int(60.0 * ln / index)

def bpm2auto(bpm, ln=10337):
    return int(60.0 * ln / bpm)

def acf(signal):
        n = len(signal)
        signal = np.array(signal)
        def r(h):
            acf_lag = ((signal[:n - h] * signal[h:]).sum()) / float(n)
            return round(acf_lag, 3)
        x = np.arange(n) # Avoiding lag 0 calculation
        acf_coeffs = map(r, x)
        return acf_coeffs
    
def beat_histogram(signal,sr=22050):
    histogram = []
    partition = _partition(signal)
    hz = [200., 400., 800., 1600., 3200.]
    for sig in partition:
        subbands = []
        #lowpass filter
        B, A = ss.butter(4,200./sr,btype='low')
        subbands.append(np.array(ss.lfilter(B,A,pywt.dwt(sig,'db4')[0])))
        #bandpass filter
        for t in range(len(hz)-1):
            B, A = ss.butter(4,[hz[t]/sr,hz[t+1]/sr],btype='bandpass')
            subbands.append(np.array(ss.lfilter(B,A,pywt.dwt(sig,'db4')[0])))
        #highpass filter
        B, A = ss.butter(4,3200./sr,btype='highpass')
        subbands.append(np.array(ss.lfilter(B,A,pywt.dwt(sig,'db4')[0])))
        #envelope extraction
        for s in range(len(subbands)):
            y = pywt.dwt(subbands[s],'db4')[0]
            y = abs(y) #full wave rectification
            y = lpfilter(y,0.99) #low pass filter
            y = y[::16] #downsampling
            subbands[s] = y - np.mean(y)#mean removal

        x = np.sum(subbands,axis=0)
        
        ac = acf(x) #enhanced autocorrelation

        pk = peak(ac[bpm2auto(200):bpm2auto(40)]) #peak finding

        histogram.append(pk)   
    return [i for row in histogram for i in row]

def _partition(signal, window_length=661500, jump=22050):
    """
    Params:
    signal: (one-dimensional array)
    window_length: (int) size of the window/frame
    jump: (int) length of distance between windows/frames
    
    Return:
    partitions: (list) list of partitioned windows/frames of length window_length and 
    """
    signal_length = len(signal)
    signal_index = range(signal_length - window_length)
    beg_index = np.array(filter(lambda x: x%jump==0, signal_index))
    end_index = beg_index + window_length
    zipped = zip(beg_index, end_index)
    
    partitions = [signal[i[0]:i[1]] for i in zipped]
    return partitions

def lpfilter(signal,alpha=0.99):
    y = range(len(signal))
    y[0] = 0
    for i in range(1,len(signal)):
        y[i] = ((1-alpha) * signal[i]) + (alpha * y[i-1])
    return y

def peak(array):
    z=np.diff(array)
    ind = []
    for i in range(len(z)-1):
        if z[i+1] < 0 and z[i] > 0:
            ind.append((array[i+1],auto2bpm(i+1+bpm2auto(200))))
    ind_ = []
    pk = []
    for k,j in sorted(ind):
        if j not in ind_:
            ind_.append(j)
            pk.append((k,j))
    return pk[-3:]

def plthis(signal):
    histogram = beat_histogram(signal)
    x = range(0,200)
    y = np.zeros((200))
    for i, j in histogram:
        y[j] = y[j] + i
    plt.xlim(60,200)
    return plt.plot(x,y)

def bh_feat(signal):
    bh = beat_histogram(signal)
    y = np.zeros((200))
    for i, j in bh:
        y[j] = y[j] + i

    r_amp = y / np.sum(y)

    a1, a0 = sorted(r_amp)[-2:]
    ra = a1 / a0
    sm = np.sum(y)
    mx = sorted(y)[-2:]
    p2, p1 = [i for i,x in enumerate(y) for j in mx if j == x]
    
    return a0, a1, ra, p1, p2, sm
#features 1 - 6

In [2]:
import cPickle

In [4]:
signal = cPickle.load(open("signal.pkl")).astype(int)

In [5]:
%%time
bh_feat(signal[:1323000])

CPU times: user 23.2 s, sys: 931 ms, total: 24.1 s
Wall time: 25 s


(0.26379934903547864,
 0.23408221626052989,
 0.88734948405444292,
 119,
 116,
 419453553.05300003)

In [6]:
%load_ext Cython

In [32]:
%%cython 
cimport numpy as np
import numpy as np
import scipy.signal as ss
import pywt
import matplotlib.pyplot as plt
import itertools as it
cimport cython

@cython.boundscheck(False)
def auto2bpm_c(int index,int ln=10337):
    return int(60.0 * ln / index)
@cython.boundscheck(False)
def bpm2auto_c(int bpm,int ln=10337):
    return int(60.0 * ln / bpm)
@cython.boundscheck(False)
def acf_c(np.ndarray[dtype = np.float64_t, ndim = 1] signal):
    cdef int n
    cdef np.ndarray[dtype = np.float64_t, ndim = 1] array
    n = len(signal)
    array = np.array([np.dot(signal[:n-i], signal[i:])/float(n) for i in xrange(n)]) 
    return array
@cython.boundscheck(False)    
def beat_histogram_c(np.ndarray[dtype = np.int64_t, ndim = 1]signal, int sr=22050):
    cdef np.ndarray[dtype = np.int64_t, ndim = 1] sig
    cdef int t, s
    histogram = []
    partition = _partition_c(signal)
    hz = [200., 400., 800., 1600., 3200.]
    for sig in partition:
        subbands = []
        #lowpass filter
        B, A = ss.butter(4,200./sr,btype='low')
        subbands.append(np.array(ss.lfilter(B,A,pywt.dwt(sig,'db4')[0])))
        #bandpass filter
        for t in xrange(len(hz)-1):
            B, A = ss.butter(4,[hz[t]/sr,hz[t+1]/sr],btype='bandpass')
            subbands.append(np.array(ss.lfilter(B,A,pywt.dwt(sig,'db4')[0])))
        #highpass filter
        B, A = ss.butter(4,3200./sr,btype='highpass')
        subbands.append(np.array(ss.lfilter(B,A,pywt.dwt(sig,'db4')[0])))
        #envelope extraction
        for s in xrange(len(subbands)):
            y = pywt.dwt(subbands[s],'db4')[0]
            y = abs(y) #full wave rectification
            y = lpfilter_c(y,0.99) #low pass filter
            y = y[::16] #downsampling
            subbands[s] = y - np.mean(y)#mean removal

        x = np.sum(subbands,axis=0)
        
        ac = acf_c(x) #enhanced autocorrelation

        pk = peak_c(ac[bpm2auto_c(200):bpm2auto_c(40)]) #peak finding

        histogram.append(pk)   
    
    return list(it.chain.from_iterable(histogram))
@cython.boundscheck(False)
def _partition_c(np.ndarray[dtype = np.int64_t , ndim = 1] signal, int window_length=661500, int jump=22050):
    """
    Params:
    signal: (one-dimensional array)
    window_length: (int) size of the window/frame
    jump: (int) length of distance between windows/frames
    
    Return:
    partitions: (list) list of partitioned windows/frames of length window_length and 
    """
    
    cdef int signal_length
    cdef np.ndarray[dtype = np.int64_t, ndim = 1] beg_index, end_index
  
    signal_length = len(signal)
    signal_index = xrange(signal_length - window_length)
    beg_index = np.array(filter(lambda x: x%jump==0, signal_index))
    end_index = beg_index + window_length
    zipped = zip(beg_index, end_index)
    
    partitions = [signal[i[0]:i[1]] for i in zipped]
    return partitions
@cython.boundscheck(False)
def lpfilter_c(np.ndarray[dtype = np.float64_t, ndim = 1] signal, float alpha=0.99):
    cdef np.ndarray[dtype = np.int64_t, ndim = 1] y
    y = np.arange(len(signal))
    y[0] = 0
    for i in xrange(1,len(signal)):
        y[i] = ((1-alpha) * signal[i]) + (alpha * y[i-1])
    return y
@cython.boundscheck(False)
def peak_c(np.ndarray[dtype = np.float64_t, ndim = 1] array):
    cdef np.ndarray[dtype = np.float64_t, ndim = 1] z 
    cdef int i, k, j 
    z = np.diff(array)
    ind = []
    for i in xrange(len(z)-1):
        if z[i+1] < 0 and z[i] > 0:
            ind.append((array[i+1],auto2bpm_c(i+1+bpm2auto_c(200))))
    ind_ = []
    pk = []
    for k,j in sorted(ind):
        if j not in ind_:
            ind_.append(j)
            pk.append((k,j))
    return pk[-3:]

@cython.boundscheck(False)
def bh_feat_c(np.ndarray[dtype = np.int64_t, ndim = 1]signal):
    cdef np.ndarray[dtype = np.float64_t, ndim = 1] y
    cdef int j
    cdef float a1, a0, ra
    bh = beat_histogram_c(signal)
    y = np.zeros((200))
    for i, j in bh:
        y[j] = y[j] + i

    r_amp = y / np.sum(y)

    a1, a0 = sorted(r_amp)[-2:]
    ra = a1 / a0
    sm = np.sum(y)
    mx = sorted(y)[-2:]

    p2, p1 = [i for i,x in enumerate(y) for j in mx if j == x]
    
    return a0, a1, ra, p1, p2, sm